In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
# import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

# 필요한 함수 정의
def make_datetime(x):
    # string 타입의 Time column을 datetime 타입으로 변경
    x     = str(x)
#     year  = int(x[:4])
#     month = int(x[4:6])
#     day   = int(x[6:8])
    hour  = int(x[8:10])
    #mim  = int(x[10:12])
    #sec  = int(x[12:])
    return hour

def string2num(x):
    # (,)( )과 같은 불필요한 데이터 정제
    x = re.sub(r"[^0-9]+", '', str(x))
    if x =='':
        return 0
    else:
        return int(x)


PATH = r'C:\Users\jeawo\PycharmProjects\lgdata/'

In [3]:
train_err  = pd.read_csv(PATH+'train_err_data.csv')
# display(train_err.head())

In [4]:
train_err['datetime'] = train_err['time'].apply(make_datetime)

train_err['datetime'] = np.where(train_err['datetime'] <= 12, 0, train_err['datetime'])
train_err['datetime'] = np.where(train_err['datetime'] > 12, 1, train_err['datetime']) 
train_err

,user_id,time,model_nm,fwver,errtype,errcode,datetime
0,10000,20201101025616,model_3,05.15.2138,15,1,0
1,10000,20201101030309,model_3,05.15.2138,12,1,0
2,10000,20201101030309,model_3,05.15.2138,11,1,0
3,10000,20201101050514,model_3,05.15.2138,16,1,0
4,10000,20201101050515,model_3,05.15.2138,4,0,0
...,...,...,...,...,...,...,...
16554658,24999,20201130163051,model_3,05.15.2138,15,1,1
16554659,24999,20201130172625,model_3,05.15.2138,16,1,1
16554660,24999,20201130172625,model_3,05.15.2138,4,0,1
16554661,24999,20201130172631,model_3,05.15.2138,4,0,1


In [21]:
train_err

,user_id,time,model_nm,fwver,errtype,errcode,datetime
0,10000,20201101025616,model_3,05.15.2138,15,1,0
1,10000,20201101030309,model_3,05.15.2138,12,1,0
2,10000,20201101030309,model_3,05.15.2138,11,1,0
3,10000,20201101050514,model_3,05.15.2138,16,1,0
4,10000,20201101050515,model_3,05.15.2138,4,0,0
...,...,...,...,...,...,...,...
16554658,24999,20201130163051,model_3,05.15.2138,15,1,1
16554659,24999,20201130172625,model_3,05.15.2138,16,1,1
16554660,24999,20201130172625,model_3,05.15.2138,4,0,1
16554661,24999,20201130172631,model_3,05.15.2138,4,0,1


In [22]:
# 데이터 설명을 확인하면
# ueser_id가 10000부터 24999까지 총 15000개가 연속적으로 존재.
train_user_id_max = 24999
train_user_id_min = 10000
train_user_number = 15000
# user_id와 errtype만을 사용하여 데이터 셋 생성
# 모든 일자에 대해서 errtype별 발생 건수를 count
# pandas의 groupby를 활용할 경우 큰 연산 자원이 필요.
# numpy로 placeholder를 만들어 구현함.
id_error = train_err[['user_id','errtype','datetime']].values
error = np.zeros((train_user_number,44))
# len(error[0])
for person_idx, err, day in tqdm(id_error):
    # person_idx - train_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    error[person_idx - train_user_id_min,err - 1] += 1
    error[person_idx - train_user_id_min,43+ day - 1] += 1
error

100%|██████████████████████████████████████████████████████████████████| 16554663/16554663 [02:18<00:00, 119719.72it/s]


array([[0.000e+00, 0.000e+00, 8.000e+00, ..., 0.000e+00, 2.290e+02,
        8.800e+01],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 1.000e+00, 5.930e+02,
        1.772e+03],
       [0.000e+00, 0.000e+00, 2.000e+00, ..., 0.000e+00, 1.480e+02,
        1.580e+02],
       ...,
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 5.000e+00, 5.080e+02,
        3.180e+02],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 9.900e+01,
        5.600e+01],
       [0.000e+00, 0.000e+00, 4.000e+00, ..., 0.000e+00, 3.790e+02,
        1.910e+02]])

In [25]:
error[0]

array([  0.,   0.,   8., 104.,   0.,   1.,   1.,   0.,   0.,   7.,  15.,
        16.,   1.,  10.,  59.,  61.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,  32.,   1.,   1.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0., 229.,  88.])

In [26]:
error.shape

(15000, 44)

In [27]:
train_prob = pd.read_csv(PATH+'train_problem_data.csv')
problem = np.zeros(15000)
# error와 동일한 방법으로 person_idx - 10000 위치에 
# person_idx의 problem이 한 번이라도 발생했다면 1
# 없다면 0
problem[train_prob.user_id.unique()-10000] = 1 
problem.shape

(15000,)

In [28]:
train_x = error
train_y = problem
print(train_x.shape)
print(train_y.shape)

(15000, 44)
(15000,)


In [10]:
train_qual = pd.read_csv(PATH+'train_quality_data.csv')
train_qual

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
1,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
2,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
3,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
4,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
828619,20201124033000,24997,04.22.1778,0.0,0,0.0,0,0,1,0,0,0,0,17,0,0
828620,20201124033000,24997,04.22.1778,0.0,0,0.0,0,0,0,0,0,0,0,17,0,0
828621,20201124033000,24997,04.22.1778,0.0,0,0.0,0,0,3,0,0,0,0,17,0,0
828622,20201124033000,24997,04.22.1778,0.0,0,0.0,0,0,0,0,0,0,0,17,0,0


In [57]:
import lightgbm as lgb
def f_pr_auc(probas_pred, y_true):
    labels=y_true.get_label()
    p, r, _ = precision_recall_curve(labels, probas_pred)
    score=auc(r,p) 
    return "pr_auc", score, True
#-------------------------------------------------------------------------------------
modelss     = []
recallss  = []
precisionss = []
auc_scoress   = []
threshold = 0.5
# 파라미터 설정
params =      {'learning_rate' : 0.003,
                'boosting_type' : 'dart',
                'objective'     : 'binary',
                'metric'        : 'auc',
                'max_depth'     : -1,
                'num_leaves'    : 100,
                'seed': 1234
                }
#-------------------------------------------------------------------------------------
# 5 Kfold cross validation
k_fold = KFold(n_splits=5, shuffle=True, random_state=12)
for train_idx, val_idx in k_fold.split(train_x):

    # split train, validation set
    X = train_x[train_idx]
    y = train_y[train_idx]
    valid_x = train_x[val_idx]
    valid_y = train_y[val_idx]

    d_train= lgb.Dataset(X, y)
    d_val  = lgb.Dataset(valid_x, valid_y)
    
    #run traning
    model = lgb.train(
                        params,
                        train_set       = d_train,
                        num_boost_round = 1000,
                        valid_sets      = d_val,
                        feval           = f_pr_auc,
                        verbose_eval    = 100, 
                        early_stopping_rounds = 5
                       )
    
    # cal valid prediction
    valid_prob = model.predict(valid_x)
    valid_pred = np.where(valid_prob > threshold, 1, 0)
    
    # cal scores
    recalls    = recall_score(    valid_y, valid_pred)
    precisions = precision_score( valid_y, valid_pred)
    auc_scores = roc_auc_score(   valid_y, valid_prob)

    # append scores
    modelss.append(model)
    recallss.append(recalls)
    precisionss.append(precisions)
    auc_scoress.append(auc_scores)

    print('='*30)

[LightGBM] [Info] Number of positive: 3989, number of negative: 8011
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003864 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4889
[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 43
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.332417 -> initscore=-0.697275
[LightGBM] [Info] Start training from score -0.697275
[100]	valid_0's auc: 0.786415	valid_0's pr_auc: 0.691092
[200]	valid_0's auc: 0.787249	valid_0's pr_auc: 0.696163
[300]	valid_0's auc: 0.787607	valid_0's pr_auc: 0.696188
[400]	valid_0's auc: 0.788738	valid_0's pr_auc: 0.696853
[500]	valid_0's auc: 0.789282	valid_0's pr_auc: 0.697217
[600]	valid_0's auc: 0.79488	valid_0's pr_auc: 0.698349
[700]	valid_0's auc: 0.797272	valid_0's pr_auc: 0.698891
[800]	valid_0's auc: 0.798452	valid_0's pr_

In [58]:
print(np.mean(auc_scoress))

0.7943260288679083


In [35]:
import xgboost as xgb 

#-------------------------------------------------------------------------------------
xgbm = XGBClassifier()
# 파라미터 설정

#-------------------------------------------------------------------------------------
k_fold = KFold(n_splits=5, shuffle=True, random_state=12)
for train_idx, val_idx in k_fold.split(train_x):

    # split train, validation set
    X = train_x[train_idx]
    y = train_y[train_idx]
    valid_x = train_x[val_idx]
    valid_y = train_y[val_idx]
    dtrain = xgb.DMatrix(data=X, label = y)
    dtest = xgb.DMatrix(data=valid_x, label=valid_y)
    xgbm_param_= {'n_estimators': 200,
                 'learning_rate': 0.01,
                 'max_depth': 20,
                 'eval_metric': 'auc'
             }
    num_rounds = 400
    # train 데이터 세트는 'train', evaluation(test) 데이터 세트는 'eval' 로 명기
    wlist = [(dtrain, 'train'), (dtest,'eval')]
    # 하이퍼 파라미터와 early stopping 파라미터를 train() 함수의 파라미터로 전달
    xgb_model = xgb.train(params = xgbm_param_, dtrain=dtrain, num_boost_round=num_rounds, evals=wlist)

[00:45:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-auc:0.96220	eval-auc:0.69748
[1]	train-auc:0.96403	eval-auc:0.69758
[2]	train-auc:0.96479	eval-auc:0.69823
[3]	train-auc:0.96590	eval-auc:0.69911
[4]	train-auc:0.96642	eval-auc:0.69999
[5]	train-auc:0.97678	eval-auc:0.71141
[6]	train-auc:0.97683	eval-auc:0.71016
[7]	train-auc:0.98004	eval-auc:0.71054
[8]	train-auc:0.98175	eval-auc:0.71386
[9]	train-auc:0.98209	eval-auc:0.71557
[10]	train-auc:0.98336	eval-auc:0.71562
[11]	train-auc:0.98408	eval-auc:0.71599
[12]	train-auc:0.98438	eval-auc:0.71614
[13]	train-auc:0.98582	eval-auc:0.71579
[14]	train-auc:0.98618	eval-auc:0.71509
[15]	train-auc:0.986

[193]	train-auc:0.99998	eval-auc:0.75853
[194]	train-auc:0.99998	eval-auc:0.75849
[195]	train-auc:0.99998	eval-auc:0.75845
[196]	train-auc:0.99998	eval-auc:0.75841
[197]	train-auc:0.99998	eval-auc:0.75843
[198]	train-auc:0.99998	eval-auc:0.75840
[199]	train-auc:0.99999	eval-auc:0.75838
[200]	train-auc:0.99999	eval-auc:0.75842
[201]	train-auc:0.99999	eval-auc:0.75838
[202]	train-auc:0.99999	eval-auc:0.75835
[203]	train-auc:0.99999	eval-auc:0.75839
[204]	train-auc:0.99999	eval-auc:0.75838
[205]	train-auc:0.99999	eval-auc:0.75840
[206]	train-auc:0.99999	eval-auc:0.75842
[207]	train-auc:0.99999	eval-auc:0.75850
[208]	train-auc:0.99999	eval-auc:0.75845
[209]	train-auc:0.99999	eval-auc:0.75838
[210]	train-auc:0.99999	eval-auc:0.75837
[211]	train-auc:0.99999	eval-auc:0.75836
[212]	train-auc:0.99999	eval-auc:0.75833
[213]	train-auc:0.99999	eval-auc:0.75844
[214]	train-auc:0.99999	eval-auc:0.75840
[215]	train-auc:0.99999	eval-auc:0.75851
[216]	train-auc:0.99999	eval-auc:0.75853
[217]	train-auc:

[393]	train-auc:1.00000	eval-auc:0.76219
[394]	train-auc:1.00000	eval-auc:0.76217
[395]	train-auc:1.00000	eval-auc:0.76215
[396]	train-auc:1.00000	eval-auc:0.76216
[397]	train-auc:1.00000	eval-auc:0.76217
[398]	train-auc:1.00000	eval-auc:0.76219
[399]	train-auc:1.00000	eval-auc:0.76219
[00:46:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-auc:0.96972	eval-auc:0.69878
[1]	train-auc:0.97189	eval-auc:0.70376
[2]	train-auc:0.97332	eval-auc:0.70381
[3]	train-auc:0.97342	eval-auc:0.70348
[4]	train-auc:0.97353	eval-auc:0.70327
[5]	train-auc:0.97482	eval-auc:0.70625
[6]	train-auc:0.97596	eval-auc:0.70771
[7]	train-auc:0.97618	eval-auc:0.70821
[8]	train-a

[186]	train-auc:0.99997	eval-auc:0.76040
[187]	train-auc:0.99997	eval-auc:0.76039
[188]	train-auc:0.99998	eval-auc:0.76043
[189]	train-auc:0.99998	eval-auc:0.76049
[190]	train-auc:0.99998	eval-auc:0.76047
[191]	train-auc:0.99998	eval-auc:0.76057
[192]	train-auc:0.99998	eval-auc:0.76059
[193]	train-auc:0.99998	eval-auc:0.76062
[194]	train-auc:0.99999	eval-auc:0.76066
[195]	train-auc:0.99999	eval-auc:0.76069
[196]	train-auc:0.99999	eval-auc:0.76085
[197]	train-auc:0.99999	eval-auc:0.76094
[198]	train-auc:0.99999	eval-auc:0.76104
[199]	train-auc:0.99999	eval-auc:0.76116
[200]	train-auc:0.99999	eval-auc:0.76125
[201]	train-auc:0.99999	eval-auc:0.76143
[202]	train-auc:0.99999	eval-auc:0.76148
[203]	train-auc:0.99999	eval-auc:0.76157
[204]	train-auc:0.99999	eval-auc:0.76165
[205]	train-auc:0.99999	eval-auc:0.76176
[206]	train-auc:0.99999	eval-auc:0.76185
[207]	train-auc:0.99999	eval-auc:0.76189
[208]	train-auc:0.99999	eval-auc:0.76188
[209]	train-auc:0.99999	eval-auc:0.76198
[210]	train-auc:

[386]	train-auc:1.00000	eval-auc:0.76894
[387]	train-auc:1.00000	eval-auc:0.76896
[388]	train-auc:1.00000	eval-auc:0.76901
[389]	train-auc:1.00000	eval-auc:0.76901
[390]	train-auc:1.00000	eval-auc:0.76900
[391]	train-auc:1.00000	eval-auc:0.76899
[392]	train-auc:1.00000	eval-auc:0.76900
[393]	train-auc:1.00000	eval-auc:0.76903
[394]	train-auc:1.00000	eval-auc:0.76904
[395]	train-auc:1.00000	eval-auc:0.76905
[396]	train-auc:1.00000	eval-auc:0.76906
[397]	train-auc:1.00000	eval-auc:0.76907
[398]	train-auc:1.00000	eval-auc:0.76907
[399]	train-auc:1.00000	eval-auc:0.76910
[00:47:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-auc:0.96616	eval-auc:0.692

...

[379]	train-auc:1.00000	eval-auc:0.75715
[380]	train-auc:1.00000	eval-auc:0.75716
[381]	train-auc:1.00000	eval-auc:0.75716
[382]	train-auc:1.00000	eval-auc:0.75717
[383]	train-auc:1.00000	eval-auc:0.75718
[384]	train-auc:1.00000	eval-auc:0.75718
[385]	train-auc:1.00000	eval-auc:0.75719
[386]	train-auc:1.00000	eval-auc:0.75722
[387]	train-auc:1.00000	eval-auc:0.75727
[388]	train-auc:1.00000	eval-auc:0.75727
[389]	train-auc:1.00000	eval-auc:0.75728
[390]	train-auc:1.00000	eval-auc:0.75730
[391]	train-auc:1.00000	eval-auc:0.75729
[392]	train-auc:1.00000	eval-auc:0.75727
[393]	train-auc:1.00000	eval-auc:0.75724
[394]	train-auc:1.00000	eval-auc:0.75723
[395]	train-auc:1.00000	eval-auc:0.75722
[396]	train-auc:1.00000	eval-auc:0.75721
[397]	train-auc:1.00000	eval-auc:0.75719
[398]	train-auc:1.00000	eval-auc:0.75724
[399]	train-auc:1.00000	eval-auc:0.75723
[00:48:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:541: 
Parameters: { n_estimators } might no

[172]	train-auc:0.99983	eval-auc:0.75659
[173]	train-auc:0.99984	eval-auc:0.75656
[174]	train-auc:0.99984	eval-auc:0.75653
[175]	train-auc:0.99985	eval-auc:0.75660
[176]	train-auc:0.99985	eval-auc:0.75655
[177]	train-auc:0.99987	eval-auc:0.75664
[178]	train-auc:0.99987	eval-auc:0.75667
[179]	train-auc:0.99988	eval-auc:0.75672
[180]	train-auc:0.99988	eval-auc:0.75681
[181]	train-auc:0.99989	eval-auc:0.75685
[182]	train-auc:0.99991	eval-auc:0.75687
[183]	train-auc:0.99992	eval-auc:0.75688
[184]	train-auc:0.99994	eval-auc:0.75688
[185]	train-auc:0.99995	eval-auc:0.75692
[186]	train-auc:0.99996	eval-auc:0.75688
[187]	train-auc:0.99996	eval-auc:0.75687
[188]	train-auc:0.99997	eval-auc:0.75686
[189]	train-auc:0.99997	eval-auc:0.75689
[190]	train-auc:0.99997	eval-auc:0.75685
[191]	train-auc:0.99998	eval-auc:0.75685
[192]	train-auc:0.99998	eval-auc:0.75682
[193]	train-auc:0.99998	eval-auc:0.75686
[194]	train-auc:0.99998	eval-auc:0.75691
[195]	train-auc:0.99998	eval-auc:0.75696
[196]	train-auc:

[372]	train-auc:1.00000	eval-auc:0.76222
[373]	train-auc:1.00000	eval-auc:0.76223
[374]	train-auc:1.00000	eval-auc:0.76223
[375]	train-auc:1.00000	eval-auc:0.76224
[376]	train-auc:1.00000	eval-auc:0.76221
[377]	train-auc:1.00000	eval-auc:0.76223
[378]	train-auc:1.00000	eval-auc:0.76225
[379]	train-auc:1.00000	eval-auc:0.76226
[380]	train-auc:1.00000	eval-auc:0.76229
[381]	train-auc:1.00000	eval-auc:0.76228
[382]	train-auc:1.00000	eval-auc:0.76229
[383]	train-auc:1.00000	eval-auc:0.76233
[384]	train-auc:1.00000	eval-auc:0.76230
[385]	train-auc:1.00000	eval-auc:0.76231
[386]	train-auc:1.00000	eval-auc:0.76230
[387]	train-auc:1.00000	eval-auc:0.76232
[388]	train-auc:1.00000	eval-auc:0.76233
[389]	train-auc:1.00000	eval-auc:0.76233
[390]	train-auc:1.00000	eval-auc:0.76236
[391]	train-auc:1.00000	eval-auc:0.76234
[392]	train-auc:1.00000	eval-auc:0.76236
[393]	train-auc:1.00000	eval-auc:0.76239
[394]	train-auc:1.00000	eval-auc:0.76242
[395]	train-auc:1.00000	eval-auc:0.76242
[396]	train-auc:

[165]	train-auc:0.99974	eval-auc:0.74485
[166]	train-auc:0.99975	eval-auc:0.74502
[167]	train-auc:0.99977	eval-auc:0.74510
[168]	train-auc:0.99978	eval-auc:0.74520
[169]	train-auc:0.99978	eval-auc:0.74524
[170]	train-auc:0.99980	eval-auc:0.74532
[171]	train-auc:0.99982	eval-auc:0.74543
[172]	train-auc:0.99984	eval-auc:0.74546
[173]	train-auc:0.99985	eval-auc:0.74554
[174]	train-auc:0.99986	eval-auc:0.74565
[175]	train-auc:0.99986	eval-auc:0.74577
[176]	train-auc:0.99988	eval-auc:0.74585
[177]	train-auc:0.99988	eval-auc:0.74600
[178]	train-auc:0.99989	eval-auc:0.74611
[179]	train-auc:0.99989	eval-auc:0.74627
[180]	train-auc:0.99989	eval-auc:0.74640
[181]	train-auc:0.99990	eval-auc:0.74641
[182]	train-auc:0.99990	eval-auc:0.74657
[183]	train-auc:0.99991	eval-auc:0.74674
[184]	train-auc:0.99991	eval-auc:0.74685
[185]	train-auc:0.99991	eval-auc:0.74699
[186]	train-auc:0.99992	eval-auc:0.74713
[187]	train-auc:0.99992	eval-auc:0.74733
[188]	train-auc:0.99993	eval-auc:0.74737
[189]	train-auc:

[365]	train-auc:1.00000	eval-auc:0.75450
[366]	train-auc:1.00000	eval-auc:0.75451
[367]	train-auc:1.00000	eval-auc:0.75452
[368]	train-auc:1.00000	eval-auc:0.75455
[369]	train-auc:1.00000	eval-auc:0.75456
[370]	train-auc:1.00000	eval-auc:0.75459
[371]	train-auc:1.00000	eval-auc:0.75461
[372]	train-auc:1.00000	eval-auc:0.75461
[373]	train-auc:1.00000	eval-auc:0.75466
[374]	train-auc:1.00000	eval-auc:0.75471
[375]	train-auc:1.00000	eval-auc:0.75469
[376]	train-auc:1.00000	eval-auc:0.75473
[377]	train-auc:1.00000	eval-auc:0.75471
[378]	train-auc:1.00000	eval-auc:0.75474
[379]	train-auc:1.00000	eval-auc:0.75479
[380]	train-auc:1.00000	eval-auc:0.75480
[381]	train-auc:1.00000	eval-auc:0.75486
[382]	train-auc:1.00000	eval-auc:0.75490
[383]	train-auc:1.00000	eval-auc:0.75492
[384]	train-auc:1.00000	eval-auc:0.75493
[385]	train-auc:1.00000	eval-auc:0.75491
[386]	train-auc:1.00000	eval-auc:0.75495
[387]	train-auc:1.00000	eval-auc:0.75496
[388]	train-auc:1.00000	eval-auc:0.75498
[389]	train-auc:

In [36]:
pred_probs = xgb_model.predict(dtest)
print('predict() 수행 결과값을 10개만 표시, 예측 확률 값으로 표시됨')
print(np.round(pred_probs[:10], 3))

# 예측 확률이 0.5보다 크면 1, 그렇지 않으면 0으로 예측값 결정해 리스트 객체인 preds에 저장
preds = [ 1 if x > 0.5 else 0 for x in pred_probs]
print('예측값 10개만 표시: ', preds[:10])

predict() 수행 결과값을 10개만 표시, 예측 확률 값으로 표시됨
[0.968 0.709 0.025 0.005 0.381 0.125 0.028 0.529 0.501 0.018]
예측값 10개만 표시:  [1, 1, 0, 0, 0, 0, 0, 1, 1, 0]


In [38]:
test_err  = pd.read_csv(PATH+'test_err_data.csv')
test_err

,user_id,time,model_nm,fwver,errtype,errcode
0,30000,20201101030227,model_1,04.16.3553,31,1
1,30000,20201101030227,model_1,04.16.3553,33,2
2,30000,20201101030228,model_1,04.16.3553,15,1
3,30000,20201101030256,model_1,04.16.3553,22,1
4,30000,20201101030300,model_1,04.16.3553,11,1
...,...,...,...,...,...,...
16532643,44998,20201130210050,model_1,04.16.3553,40,0
16532644,44998,20201130211831,model_1,04.16.3553,31,1
16532645,44998,20201130211832,model_1,04.16.3553,15,1
16532646,44998,20201130212259,model_1,04.16.3553,16,1


In [39]:
test_err['datetime'] = test_err['time'].apply(make_datetime)

test_err['datetime'] = np.where(test_err['datetime'] <= 12, 0, test_err['datetime'])
test_err['datetime'] = np.where(test_err['datetime'] > 12, 1, test_err['datetime']) 
test_err

,user_id,time,model_nm,fwver,errtype,errcode,datetime
0,30000,20201101030227,model_1,04.16.3553,31,1,0
1,30000,20201101030227,model_1,04.16.3553,33,2,0
2,30000,20201101030228,model_1,04.16.3553,15,1,0
3,30000,20201101030256,model_1,04.16.3553,22,1,0
4,30000,20201101030300,model_1,04.16.3553,11,1,0
...,...,...,...,...,...,...,...
16532643,44998,20201130210050,model_1,04.16.3553,40,0,1
16532644,44998,20201130211831,model_1,04.16.3553,31,1,1
16532645,44998,20201130211832,model_1,04.16.3553,15,1,1
16532646,44998,20201130212259,model_1,04.16.3553,16,1,1


In [40]:
# 데이터 설명을 확인하면
# test 데이터는 ueser_id가 30000부터 44998까지 총 14999개가 존재.
test_user_id_max = 44998
test_user_id_min = 30000
test_user_number = 14999
id_error_t = test_err[['user_id','errtype','datetime']].values
test_x  = np.zeros((test_user_number,44))
# len(error[0])
for person_idx, err, days in tqdm(id_error_t):
    # person_idx - train_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    test_x [person_idx - test_user_id_min,err - 1] += 1
    test_x [person_idx - test_user_id_min,43+ days - 1] += 1
test_x 

100%|██████████████████████████████████████████████████████████████████| 16532648/16532648 [02:17<00:00, 120472.81it/s]


array([[0.000e+00, 0.000e+00, 0.000e+00, ..., 2.000e+00, 6.350e+02,
        2.115e+03],
       [0.000e+00, 0.000e+00, 3.000e+00, ..., 0.000e+00, 1.700e+02,
        1.140e+02],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 2.000e+00, 5.140e+02,
        4.270e+02],
       ...,
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 3.000e+00, 8.700e+02,
        1.363e+03],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 9.981e+03,
        1.469e+04],
       [0.000e+00, 0.000e+00, 0.000e+00, ..., 0.000e+00, 5.830e+02,
        2.900e+02]])

In [42]:
# 혼동행렬, 정확도, 정밀도, 재현율, F1, AUC 불러오기
def get_clf_eval(y_test, y_pred):
    confusion = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred)
    AUC = roc_auc_score(y_test, y_pred)
    print('오차행렬:\n', confusion)
    print('\n정확도: {:.4f}'.format(accuracy))
    print('정밀도: {:.4f}'.format(precision))
    print('재현율: {:.4f}'.format(recall))
    print('F1: {:.4f}'.format(F1))
    print('AUC: {:.4f}'.format(AUC))
get_clf_eval(valid_y, preds)

오차행렬:
 [[1678  321]
 [ 490  511]]

정확도: 0.7297
정밀도: 0.6142
재현율: 0.5105
F1: 0.5576
AUC: 0.6750


In [53]:
# max_depth = 3, 학습률은 0.1, 예제가 이진분류이므로 목적함수(objective)는 binary:logistic(이진 로지스틱)
# 부스팅 반복횟수는 400

from xgboost import XGBClassifier
models     = []
recalls    = []
precisions = []
auc_scores   = []
threshold = 0.5
k_fold = KFold(n_splits=5, shuffle=True, random_state=132)
for train_idx, val_idx in k_fold.split(train_x):
    # split train, validation set
    X = train_x[train_idx]
    y = train_y[train_idx]
    valid_x = train_x[val_idx]
    valid_y = train_y[val_idx]
    evals = [(valid_x,valid_y)]
    xgb_wrapper = XGBClassifier(n_estimators = 400, learning_rate = 0.01, max_depth = 20)
    # train 데이터 세트는 'train', evaluation(test) 데이터 세트는 'eval' 로 명기
    xgb_wrapper.fit(train_x, train_y,early_stopping_rounds = 10,eval_metric="auc", eval_set = evals, verbose=True)
    # 하이퍼 파라미터와 early stopping 파라미터를 train() 함수의 파라미터로 전달
    # cal valid prediction
    valid_prob = xgb_wrapper.predict(valid_x)
    valid_pred = np.where(valid_prob > threshold, 1, 0)
    
    # cal scores
    recall    = recall_score(    valid_y, valid_pred)
    precision = precision_score( valid_y, valid_pred)
    auc_score = roc_auc_score(   valid_y, valid_prob)

    # append scores
    models.append(model)
    recalls.append(recall)
    precisions.append(precision)
    auc_scores.append(auc_score)

    print('==========================================================')

[0]	validation_0-auc:0.91623
[1]	validation_0-auc:0.92061
[2]	validation_0-auc:0.92021
[3]	validation_0-auc:0.92249
[4]	validation_0-auc:0.92246
[5]	validation_0-auc:0.92469
[6]	validation_0-auc:0.92464
[7]	validation_0-auc:0.92687
[8]	validation_0-auc:0.92814
[9]	validation_0-auc:0.92980
[10]	validation_0-auc:0.93045
[11]	validation_0-auc:0.93249
[12]	validation_0-auc:0.93305
[13]	validation_0-auc:0.93550
[14]	validation_0-auc:0.93757
[15]	validation_0-auc:0.93898
[16]	validation_0-auc:0.94048
[17]	validation_0-auc:0.94141
[18]	validation_0-auc:0.94306
[19]	validation_0-auc:0.94358
[20]	validation_0-auc:0.94465
[21]	validation_0-auc:0.94546
[22]	validation_0-auc:0.94626
[23]	validation_0-auc:0.94684
[24]	validation_0-auc:0.94818
[25]	validation_0-auc:0.94891
[26]	validation_0-auc:0.94937
[27]	validation_0-auc:0.95010
[28]	validation_0-auc:0.95043
[29]	validation_0-auc:0.95131
[30]	validation_0-auc:0.95175
[31]	validation_0-auc:0.95267
[32]	validation_0-auc:0.95385
[33]	validation_0-au

[268]	validation_0-auc:0.99772
[269]	validation_0-auc:0.99774
[270]	validation_0-auc:0.99776
[271]	validation_0-auc:0.99779
[272]	validation_0-auc:0.99781
[273]	validation_0-auc:0.99781
[274]	validation_0-auc:0.99782
[275]	validation_0-auc:0.99785
[276]	validation_0-auc:0.99788
[277]	validation_0-auc:0.99790
[278]	validation_0-auc:0.99791
[279]	validation_0-auc:0.99793
[280]	validation_0-auc:0.99794
[281]	validation_0-auc:0.99796
[282]	validation_0-auc:0.99799
[283]	validation_0-auc:0.99799
[284]	validation_0-auc:0.99800
[285]	validation_0-auc:0.99802
[286]	validation_0-auc:0.99803
[287]	validation_0-auc:0.99803
[288]	validation_0-auc:0.99805
[289]	validation_0-auc:0.99805
[290]	validation_0-auc:0.99805
[291]	validation_0-auc:0.99806
[292]	validation_0-auc:0.99809
[293]	validation_0-auc:0.99809
[294]	validation_0-auc:0.99811
[295]	validation_0-auc:0.99813
[296]	validation_0-auc:0.99814
[297]	validation_0-auc:0.99816
[298]	validation_0-auc:0.99817
[299]	validation_0-auc:0.99818
[300]	va

[134]	validation_0-auc:0.98846
[135]	validation_0-auc:0.98864
[136]	validation_0-auc:0.98880
[137]	validation_0-auc:0.98893
[138]	validation_0-auc:0.98911
[139]	validation_0-auc:0.98923
[140]	validation_0-auc:0.98933
[141]	validation_0-auc:0.98950
[142]	validation_0-auc:0.98966
[143]	validation_0-auc:0.98983
[144]	validation_0-auc:0.98995
[145]	validation_0-auc:0.99010
[146]	validation_0-auc:0.99025
[147]	validation_0-auc:0.99036
[148]	validation_0-auc:0.99049
[149]	validation_0-auc:0.99063
[150]	validation_0-auc:0.99074
[151]	validation_0-auc:0.99087
[152]	validation_0-auc:0.99099
[153]	validation_0-auc:0.99112
[154]	validation_0-auc:0.99125
[155]	validation_0-auc:0.99136
[156]	validation_0-auc:0.99148
[157]	validation_0-auc:0.99159
[158]	validation_0-auc:0.99176
[159]	validation_0-auc:0.99191
[160]	validation_0-auc:0.99205
[161]	validation_0-auc:0.99215
[162]	validation_0-auc:0.99226
[163]	validation_0-auc:0.99235
[164]	validation_0-auc:0.99246
[165]	validation_0-auc:0.99255
[166]	va

[399]	validation_0-auc:0.99973
[0]	validation_0-auc:0.91363
[1]	validation_0-auc:0.91613
[2]	validation_0-auc:0.91623
[3]	validation_0-auc:0.91766
[4]	validation_0-auc:0.91743
[5]	validation_0-auc:0.91821
[6]	validation_0-auc:0.91820
[7]	validation_0-auc:0.92027
[8]	validation_0-auc:0.92196
[9]	validation_0-auc:0.92340
[10]	validation_0-auc:0.92465
[11]	validation_0-auc:0.92641
[12]	validation_0-auc:0.92754
[13]	validation_0-auc:0.92940
[14]	validation_0-auc:0.93123
[15]	validation_0-auc:0.93231
[16]	validation_0-auc:0.93390
[17]	validation_0-auc:0.93479
[18]	validation_0-auc:0.93678
[19]	validation_0-auc:0.93775
[20]	validation_0-auc:0.93931
[21]	validation_0-auc:0.93998
[22]	validation_0-auc:0.94095
[23]	validation_0-auc:0.94140
[24]	validation_0-auc:0.94210
[25]	validation_0-auc:0.94303
[26]	validation_0-auc:0.94337
[27]	validation_0-auc:0.94404
[28]	validation_0-auc:0.94435
[29]	validation_0-auc:0.94512
[30]	validation_0-auc:0.94572
[31]	validation_0-auc:0.94650
[32]	validation_0-a

[265]	validation_0-auc:0.99764
[266]	validation_0-auc:0.99768
[267]	validation_0-auc:0.99772
[268]	validation_0-auc:0.99776
[269]	validation_0-auc:0.99779
[270]	validation_0-auc:0.99783
[271]	validation_0-auc:0.99786
[272]	validation_0-auc:0.99789
[273]	validation_0-auc:0.99791
[274]	validation_0-auc:0.99795
[275]	validation_0-auc:0.99799
[276]	validation_0-auc:0.99803
[277]	validation_0-auc:0.99805
[278]	validation_0-auc:0.99808
[279]	validation_0-auc:0.99811
[280]	validation_0-auc:0.99814
[281]	validation_0-auc:0.99816
[282]	validation_0-auc:0.99819
[283]	validation_0-auc:0.99822
[284]	validation_0-auc:0.99825
[285]	validation_0-auc:0.99829
[286]	validation_0-auc:0.99831
[287]	validation_0-auc:0.99833
[288]	validation_0-auc:0.99837
[289]	validation_0-auc:0.99838
[290]	validation_0-auc:0.99841
[291]	validation_0-auc:0.99843
[292]	validation_0-auc:0.99847
[293]	validation_0-auc:0.99850
[294]	validation_0-auc:0.99853
[295]	validation_0-auc:0.99855
[296]	validation_0-auc:0.99857
[297]	va

[131]	validation_0-auc:0.99054
[132]	validation_0-auc:0.99069
[133]	validation_0-auc:0.99085
[134]	validation_0-auc:0.99098
[135]	validation_0-auc:0.99112
[136]	validation_0-auc:0.99131
[137]	validation_0-auc:0.99139
[138]	validation_0-auc:0.99162
[139]	validation_0-auc:0.99173
[140]	validation_0-auc:0.99191
[141]	validation_0-auc:0.99203
[142]	validation_0-auc:0.99222
[143]	validation_0-auc:0.99233
[144]	validation_0-auc:0.99249
[145]	validation_0-auc:0.99267
[146]	validation_0-auc:0.99283
[147]	validation_0-auc:0.99294
[148]	validation_0-auc:0.99309
[149]	validation_0-auc:0.99323
[150]	validation_0-auc:0.99337
[151]	validation_0-auc:0.99345
[152]	validation_0-auc:0.99355
[153]	validation_0-auc:0.99364
[154]	validation_0-auc:0.99372
[155]	validation_0-auc:0.99379
[156]	validation_0-auc:0.99390
[157]	validation_0-auc:0.99399
[158]	validation_0-auc:0.99408
[159]	validation_0-auc:0.99419
[160]	validation_0-auc:0.99430
[161]	validation_0-auc:0.99440
[162]	validation_0-auc:0.99447
[163]	va

[396]	validation_0-auc:0.99981
[397]	validation_0-auc:0.99981
[398]	validation_0-auc:0.99981
[399]	validation_0-auc:0.99981
[0]	validation_0-auc:0.90893
[1]	validation_0-auc:0.91104
[2]	validation_0-auc:0.91027
[3]	validation_0-auc:0.91173
[4]	validation_0-auc:0.91227
[5]	validation_0-auc:0.91248
[6]	validation_0-auc:0.91245
[7]	validation_0-auc:0.91807
[8]	validation_0-auc:0.91919
[9]	validation_0-auc:0.92061
[10]	validation_0-auc:0.92268
[11]	validation_0-auc:0.92517
[12]	validation_0-auc:0.92652
[13]	validation_0-auc:0.92912
[14]	validation_0-auc:0.93037
[15]	validation_0-auc:0.93180
[16]	validation_0-auc:0.93321
[17]	validation_0-auc:0.93421
[18]	validation_0-auc:0.93544
[19]	validation_0-auc:0.93614
[20]	validation_0-auc:0.93750
[21]	validation_0-auc:0.93853
[22]	validation_0-auc:0.93973
[23]	validation_0-auc:0.94053
[24]	validation_0-auc:0.94162
[25]	validation_0-auc:0.94251
[26]	validation_0-auc:0.94294
[27]	validation_0-auc:0.94373
[28]	validation_0-auc:0.94406
[29]	validation_

[262]	validation_0-auc:0.99878
[263]	validation_0-auc:0.99881
[264]	validation_0-auc:0.99883
[265]	validation_0-auc:0.99886
[266]	validation_0-auc:0.99888
[267]	validation_0-auc:0.99890
[268]	validation_0-auc:0.99892
[269]	validation_0-auc:0.99895
[270]	validation_0-auc:0.99896
[271]	validation_0-auc:0.99899
[272]	validation_0-auc:0.99901
[273]	validation_0-auc:0.99903
[274]	validation_0-auc:0.99904
[275]	validation_0-auc:0.99906
[276]	validation_0-auc:0.99907
[277]	validation_0-auc:0.99909
[278]	validation_0-auc:0.99911
[279]	validation_0-auc:0.99913
[280]	validation_0-auc:0.99914
[281]	validation_0-auc:0.99916
[282]	validation_0-auc:0.99918
[283]	validation_0-auc:0.99919
[284]	validation_0-auc:0.99920
[285]	validation_0-auc:0.99921
[286]	validation_0-auc:0.99923
[287]	validation_0-auc:0.99925
[288]	validation_0-auc:0.99926
[289]	validation_0-auc:0.99927
[290]	validation_0-auc:0.99929
[291]	validation_0-auc:0.99930
[292]	validation_0-auc:0.99931
[293]	validation_0-auc:0.99932
[294]	va

In [54]:
# 예측 결과 확인
print(np.mean(auc_scores))

0.9847078607984123


In [59]:
# 예측
pred_y_lists = []
for model in modelss:
    pred_y = model.predict(test_x)
    pred_y_lists.append(pred_y.reshape(-1,1))
    
pred_ensembles = np.mean(pred_y_lists, axis = 0)
pred_ensembles

array([[0.67727861],
       [0.33167102],
       [0.40053494],
       ...,
       [0.58887859],
       [0.70723123],
       [0.43344446]])

In [60]:
sample_submssion = pd.read_csv(PATH+'sample_submission.csv')
sample_submssion['problem'] = pred_ensembles.reshape(-1)
sample_submssion.to_csv("dacon_lg_lgb.csv", index = False)
sample_submssion

,user_id,problem
0,30000,0.677279
1,30001,0.331671
2,30002,0.400535
3,30003,0.575250
4,30004,0.552058
...,...,...
14994,44994,0.423424
14995,44995,0.391156
14996,44996,0.588879
14997,44997,0.707231
